In [ ]:
# dev/inference.ipynb
import mlflow
import pandas as pd

print("DEV INFERENCE NOTEBOOK")

model_name = "my_classification_model_dev"  # dev-specific model name
model_uri = f"models:/{model_name}/Staging" # dev might store in 'Staging' or 'None'

model = mlflow.pyfunc.load_model(model_uri)

df = pd.DataFrame({
    "feature1": [5.1, 4.9],
    "feature2": [3.5, 3.0],
    "feature3": [1.4, 1.4],
    "feature4": [0.2, 0.2]
})

predictions = model.predict(df)
print("Dev inference predictions:", predictions)
